# Massachusetts ER Wait Time Analysis

## 01 — Data Cleaning

This notebook cleans the raw CHIA ED length-of-stay (LOS) dataset and outputs a cleaned file to `data/processed/df_clean.csv` for downstream analysis.

> **Note:** Raw/processed data is ignored by git. Place raw files in `data/raw/` as described in the README.

## Setup/Imports

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

In [3]:
df = pd.read_excel("LOS_data.xlsx",sheet_name = 'II-1', skiprows = 7)
df

,Year (FFY),Month,Emergency Department,Visits,Percent of Total Visits within Month,Average Length of Stay (Hours)
0,2019,October 2018,All Visits,202625,100.00%,4.239666
1,2019,October 2018,Anna Jaques Hospital,1629,0.80%,2.899785
2,2019,October 2018,Athol Memorial Hospital,854,0.42%,3.469106
3,2019,October 2018,Baystate Franklin Medical Center,1709,0.84%,3.660659
4,2019,October 2018,Baystate Medical Center,7025,3.46%,5.606492
...,...,...,...,...,...,...
4443,2021,February 2021,Fairview Hospital,628,0.48%,2.060828
4444,2020,March 2020,Mercy Medical Center,4199,2.67%,2.0503
4445,2020,March 2020,Massachusetts Eye and Ear Infirmary,1270,0.80%,2.028989
4446,2020,June 2020,Massachusetts Eye and Ear Infirmary,1390,1.00%,1.947998


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4448 entries, 0 to 4447
Data columns (total 6 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   Year (FFY)                            4448 non-null   int64 
 1   Month                                 4448 non-null   object
 2   Emergency Department                  4448 non-null   object
 3   Visits                                4448 non-null   int64 
 4   Percent of Total Visits within Month  4448 non-null   object
 5   Average Length of Stay (Hours)        4448 non-null   object
dtypes: int64(2), object(4)
memory usage: 208.6+ KB


## Data Cleaning

In [5]:
# change column names for ease of analysis
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ','_')

#rename columns
df.rename(columns = {'average_length_of_stay_(hours)':'average_los_hours'}, inplace = True)
df.rename(columns = {'percent_of_total_visits_within_month':'pct_monthly_visits'}, inplace = True)

#change datatypes
df['month'] = pd.to_datetime(df['month'], format="%B %Y")
df['visits'] = pd.to_numeric(df['visits'])
df['average_los_hours'] = pd.to_numeric(df['average_los_hours'], errors = 'coerce')

#convert pct to a float
df["pct_monthly_visits"] = (
    df["pct_monthly_visits"]
    .str.replace('%', '', regex=False)
    .astype(float) / 100
)

In [6]:
df['emergency_department'].nunique()

59

In [7]:
# assign hospital system to each hospital (found online)

system_map = {
    "Massachusetts General Hospital": "Mass General Brigham",
    "Brigham and Women's Hospital": "Mass General Brigham",
    "Newton-Wellesley Hospital": "Mass General Brigham",
    "Cooley Dickinson Hospital": "Mass General Brigham",
    "Massachusetts Eye and Ear Infirmary": "Mass General Brigham",
    "Beth Israel Deaconess Medical Center": "Beth Israel Lahey Health",
    "Lahey Hospital & Medical Center": "Beth Israel Lahey Health",
    "Beth Israel Deaconess Hospital - Milton": "Beth Israel Lahey Health",
    "Beth Israel Deaconess Hospital - Needham": "Beth Israel Lahey Health",
    "Beth Israel Deaconess Hospital - Plymouth": "Beth Israel Lahey Health",
    "Mount Auburn Hospital": "Beth Israel Lahey Health",
    "Tufts Medical Center": "Tufts Medicine",
    "UMass Memorial Medical Center": "UMass Memorial Health",
    "Marlborough Hospital": "UMass Memorial Health",
    "HealthAlliance-Clinton Hospital": "UMass Memorial Health",
    "Baystate Medical Center": "Baystate Health",
    "Baystate Franklin Medical Center": "Baystate Health",
    "Baystate Noble Hospital": "Baystate Health",
    "Baystate Wing Hospital": "Baystate Health",
    "Southcoast Hospitals Group": "Southcoast Health",
    "South Shore Hospital": "South Shore Health",
    "Cape Cod Hospital": "Cape Cod Healthcare",
    "Falmouth Hospital": "Cape Cod Healthcare",
    "Cambridge Health Alliance": "CHA",
    "Boston Medical Center": "BMC Health System",
    "Boston Children's Hospital": "Boston Children’s",
    "Lawrence General Hospital": "Independent",
    "Sturdy Memorial Hospital": "Independent",
    "Emerson Hospital": "Independent",
    "Nantucket Cottage Hospital": "Mass General Brigham",
    "Martha's Vineyard Hospital": "Mass General Brigham",
    "North Shore Medical Center": "Mass General Brigham",
    "Lahey Hospital & Medical Center": "Beth Israel Lahey Health",
    "Signature Healthcare Brockton Hospital": "Signature Healthcare",
    "Saint Vincent Hospital": "Tenet Healthcare",
    "MetroWest Medical Center": "Tenet Healthcare",
    "Holy Family Hospital": "Steward Health Care",
    "Saint Anne's Hospital": "Steward Health Care",
    "Steward Carney Hospital": "Steward Health Care",
    "Steward Norwood Hospital": "Steward Health Care",
    "Morton Hospital": "Steward Health Care",
    "Northeast Hospital": "Beth Israel Lahey Health",  # legacy name
    "Heywood Hospital": "Independent",
    "Harrington Memorial Hospital": "Independent",
    "Fairview Hospital": "Berkshire Health Systems",
    "Berkshire Medical Center": "Berkshire Health Systems",
    "Milford Regional Medical Center": "Independent",
    "MelroseWakefield Hospital": "Tufts Medicine",
    "Anna Jaques Hospital": "Beth Israel Lahey Health",
    "Nashoba Valley Medical Center": "Prime Healthcare",
    "Athol Memorial Hospital": "Heywood Healthcare",
    "Boston Medical Center - Brighton": "Boston Medical Center Health System",
    "Boston Medical Center - South": "Boston Medical Center Health System",
    "Brigham and Women's Faulkner Hospital": "Mass General Brigham",
    "Holyoke Medical Center": "Valley Health Systems (Independent)",
    "Lowell General Hospital": "Tufts Medicine",
    "Mercy Medical Center": "Trinity Health",
    "Winchester Hospital": "Beth Israel Lahey Health",
    "North Adams Regional Hospital": "Closed/Legacy (Berkshire Health Systems)"
}

df["hospital_system"] = df["emergency_department"].map(system_map)

# See any hospitals not mapped yet:
unmapped = df.loc[df["hospital_system"].isna(), "emergency_department"].unique()
unmapped

array(['All Visits'], dtype=object)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4448 entries, 0 to 4447
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   year_(ffy)            4448 non-null   int64         
 1   month                 4448 non-null   datetime64[ns]
 2   emergency_department  4448 non-null   object        
 3   visits                4448 non-null   int64         
 4   pct_monthly_visits    4448 non-null   float64       
 5   average_los_hours     4400 non-null   float64       
 6   hospital_system       4370 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 243.4+ KB


In [10]:
# location csv generated from  chatgpt 
df_loc = pd.read_csv('/Users/chrisbice/Desktop/Data Projects/ER Wait-Time Analysis/ma_hospital_locations_complete.csv')

In [11]:
df_loc

,emergency_department,city,latitude,longitude,system_name,region
0,Anna Jaques Hospital,Newburyport,42.8120,-70.8717,Beth Israel Lahey Health,North Shore
1,Athol Memorial Hospital,Athol,42.5953,-72.2266,Heywood Healthcare,Central MA
2,Baystate Franklin Medical Center,Greenfield,42.5879,-72.6007,Baystate Health,Western MA
3,Baystate Noble Hospital,Westfield,42.1215,-72.7600,Baystate Health,Western MA
4,Baystate Wing Hospital,Palmer,42.1584,-72.3306,Baystate Health,Western MA
5,Berkshire Medical Center,Pittsfield,42.4534,-73.2540,Berkshire Health Systems,Western MA
6,Beth Israel Deaconess Hospital - Milton,Milton,42.2495,-71.0656,Beth Israel Lahey Health,South Shore
7,Beth Israel Deaconess Hospital - Needham,Needham,42.2816,-71.2410,Beth Israel Lahey Health,Boston Metro
8,Beth Israel Deaconess Hospital - Plymouth,Plymouth,41.9570,-70.6790,Beth Israel Lahey Health,South Shore
9,Beth Israel Deaconess Medical Center,Boston,42.3370,-71.1057,Beth Israel Lahey Health,Boston Metro


In [12]:
# merge data frames 
df = df.merge(df_loc, on = 'emergency_department', how = 'left')
df

,year_(ffy),month,emergency_department,visits,pct_monthly_visits,average_los_hours,hospital_system,city,latitude,longitude,system_name,region
0,2019,2018-10-01,All Visits,202625,1.0000,4.239666,NaN,NaN,NaN,NaN,NaN,NaN
1,2019,2018-10-01,Anna Jaques Hospital,1629,0.0080,2.899785,Beth Israel Lahey Health,Newburyport,42.8120,-70.8717,Beth Israel Lahey Health,North Shore
2,2019,2018-10-01,Athol Memorial Hospital,854,0.0042,3.469106,Heywood Healthcare,Athol,42.5953,-72.2266,Heywood Healthcare,Central MA
3,2019,2018-10-01,Baystate Franklin Medical Center,1709,0.0084,3.660659,Baystate Health,Greenfield,42.5879,-72.6007,Baystate Health,Western MA
4,2019,2018-10-01,Baystate Medical Center,7025,0.0346,5.606492,Baystate Health,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4443,2021,2021-02-01,Fairview Hospital,628,0.0048,2.060828,Berkshire Health Systems,Great Barrington,42.1968,-73.3634,Berkshire Health Systems,Western MA
4444,2020,2020-03-01,Mercy Medical Center,4199,0.0267,2.050300,Trinity Health,Springfield,42.1029,-72.5787,Trinity Health,Western MA
4445,2020,2020-03-01,Massachusetts Eye and Ear Infirmary,1270,0.0080,2.028989,Mass General Brigham,Boston,42.3630,-71.0688,Mass General Brigham,Boston Metro
4446,2020,2020-06-01,Massachusetts Eye and Ear Infirmary,1390,0.0100,1.947998,Mass General Brigham,Boston,42.3630,-71.0688,Mass General Brigham,Boston Metro


In [13]:
# drop redundant column 'system_name' and fiscal year
df.drop('system_name', axis = 1,inplace = True)
df.drop('year_(ffy)',axis = 1,inplace = True)


In [14]:
# identify entries with missing geolocation
missing_geo = df[(df['city'].isnull()) & (df['emergency_department'] != 'All Visits')]
missing_geo['emergency_department'].unique()

array(['Baystate Medical Center', 'HealthAlliance-Clinton Hospital',
       'Marlborough Hospital', 'Massachusetts General Hospital'],
      dtype=object)

In [15]:
#manually fill in the missing hospital data
df.loc[df['emergency_department'] == "Baystate Medical Center", ['city', 'latitude', 'longitude', 'region']] = [
    "Springfield", 42.12171362154685, -72.60298297011474, "Western MA"

]
df.loc[df['emergency_department'] == "HealthAlliance-Clinton Hospital", ['city', 'latitude', 'longitude', 'region']] = [
    "Clinton", 42.427672202512646, -71.69325696064523, "Central MA"

]
df.loc[df['emergency_department'] == "Marlborough Hospital", ['city', 'latitude', 'longitude', 'region']] = [
    "Clinton", 42.35488421278838, -71.55505223181254, "MetroWest"

]
df.loc[df['emergency_department'] == "Massachusetts General Hospital", ['city', 'latitude', 'longitude', 'region']] = [
    "Boston", 42.362607817712124, -71.06838040221002, "Boston Metro"

]


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4448 entries, 0 to 4447
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   month                 4448 non-null   datetime64[ns]
 1   emergency_department  4448 non-null   object        
 2   visits                4448 non-null   int64         
 3   pct_monthly_visits    4448 non-null   float64       
 4   average_los_hours     4400 non-null   float64       
 5   hospital_system       4370 non-null   object        
 6   city                  4370 non-null   object        
 7   latitude              4370 non-null   float64       
 8   longitude             4370 non-null   float64       
 9   region                4370 non-null   object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(4)
memory usage: 347.6+ KB


In [17]:
df[df['average_los_hours'].isna()]

,month,emergency_department,visits,pct_monthly_visits,average_los_hours,hospital_system,city,latitude,longitude,region
697,2019-10-01,MetroWest Medical Center,3727,0.0187,NaN,Tenet Healthcare,Framingham,42.2979,-71.4372,MetroWest
698,2019-10-01,Saint Vincent Hospital,4767,0.0239,NaN,Tenet Healthcare,Worcester,42.2614,-71.8022,Central MA
699,2019-11-01,MetroWest Medical Center,3517,0.0189,NaN,Tenet Healthcare,Framingham,42.2979,-71.4372,MetroWest
700,2019-11-01,Saint Vincent Hospital,4356,0.0234,NaN,Tenet Healthcare,Worcester,42.2614,-71.8022,Central MA
701,2019-12-01,MetroWest Medical Center,3967,0.0201,NaN,Tenet Healthcare,Framingham,42.2979,-71.4372,MetroWest
702,2019-12-01,Saint Vincent Hospital,4428,0.0225,NaN,Tenet Healthcare,Worcester,42.2614,-71.8022,Central MA
703,2020-01-01,MetroWest Medical Center,4116,0.0192,NaN,Tenet Healthcare,Framingham,42.2979,-71.4372,MetroWest
704,2020-01-01,Saint Vincent Hospital,5032,0.0234,NaN,Tenet Healthcare,Worcester,42.2614,-71.8022,Central MA
705,2020-02-01,MetroWest Medical Center,3596,0.0185,NaN,Tenet Healthcare,Framingham,42.2979,-71.4372,MetroWest
706,2020-02-01,Saint Vincent Hospital,4518,0.0232,NaN,Tenet Healthcare,Worcester,42.2614,-71.8022,Central MA


In [18]:
# Make a safe copy
df_filled = df.copy()

# Hospitals with missing LOS
target_hospitals = [
    "MetroWest Medical Center",
    "Saint Vincent Hospital"
]

for hospital in target_hospitals:
    
    # Subset hospital data
    hosp_df = df_filled[df_filled["emergency_department"] == hospital].copy()
    
    # Split into known and missing LOS
    train = hosp_df[hosp_df["average_los_hours"].notna()]
    missing = hosp_df[hosp_df["average_los_hours"].isna()]
    
    # Skip if no missing values
    if len(missing) == 0:
        continue
    
    # -------------------------------
    # Train regression: LOS ~ Visits
    # -------------------------------
    X_train = train[["visits"]]
    y_train = train["average_los_hours"]
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # -------------------------------
    # Predict missing LOS
    # -------------------------------
    X_missing = missing[["visits"]]
    predicted_los = model.predict(X_missing)
    
    # -------------------------------
    # Fill values back into main DF
    # -------------------------------
    df.loc[
        (df["emergency_department"] == hospital) &
        (df["average_los_hours"].isna()),
        "average_los_hours"
    ] = predicted_los

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4448 entries, 0 to 4447
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   month                 4448 non-null   datetime64[ns]
 1   emergency_department  4448 non-null   object        
 2   visits                4448 non-null   int64         
 3   pct_monthly_visits    4448 non-null   float64       
 4   average_los_hours     4448 non-null   float64       
 5   hospital_system       4370 non-null   object        
 6   city                  4370 non-null   object        
 7   latitude              4370 non-null   float64       
 8   longitude             4370 non-null   float64       
 9   region                4370 non-null   object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(4)
memory usage: 347.6+ KB


In [19]:
df.loc[697:701]

,month,emergency_department,visits,pct_monthly_visits,average_los_hours,hospital_system,city,latitude,longitude,region
697,2019-10-01,MetroWest Medical Center,3727,0.0187,4.343740,Tenet Healthcare,Framingham,42.2979,-71.4372,MetroWest
698,2019-10-01,Saint Vincent Hospital,4767,0.0239,4.779541,Tenet Healthcare,Worcester,42.2614,-71.8022,Central MA
699,2019-11-01,MetroWest Medical Center,3517,0.0189,4.567416,Tenet Healthcare,Framingham,42.2979,-71.4372,MetroWest
700,2019-11-01,Saint Vincent Hospital,4356,0.0234,4.946924,Tenet Healthcare,Worcester,42.2614,-71.8022,Central MA
701,2019-12-01,MetroWest Medical Center,3967,0.0201,4.088110,Tenet Healthcare,Framingham,42.2979,-71.4372,MetroWest


In [20]:
corrs = (
    df.groupby("emergency_department")[["visits", "average_los_hours"]]
      .corr()
      .iloc[0::2, -1]             # extract the correlation values
      .rename("correlation")
)

# merge back into your main dataframe
df = df.merge(corrs, on="emergency_department", how="left")

df

,month,emergency_department,visits,pct_monthly_visits,average_los_hours,hospital_system,city,latitude,longitude,region,correlation
0,2018-10-01,All Visits,202625,1.0000,4.239666,NaN,NaN,NaN,NaN,NaN,-0.094358
1,2018-10-01,Anna Jaques Hospital,1629,0.0080,2.899785,Beth Israel Lahey Health,Newburyport,42.812000,-70.871700,North Shore,-0.275977
2,2018-10-01,Athol Memorial Hospital,854,0.0042,3.469106,Heywood Healthcare,Athol,42.595300,-72.226600,Central MA,0.104899
3,2018-10-01,Baystate Franklin Medical Center,1709,0.0084,3.660659,Baystate Health,Greenfield,42.587900,-72.600700,Western MA,-0.141135
4,2018-10-01,Baystate Medical Center,7025,0.0346,5.606492,Baystate Health,Springfield,42.121714,-72.602983,Western MA,-0.100213
...,...,...,...,...,...,...,...,...,...,...,...
4443,2021-02-01,Fairview Hospital,628,0.0048,2.060828,Berkshire Health Systems,Great Barrington,42.196800,-73.363400,Western MA,0.476108
4444,2020-03-01,Mercy Medical Center,4199,0.0267,2.050300,Trinity Health,Springfield,42.102900,-72.578700,Western MA,-0.762083
4445,2020-03-01,Massachusetts Eye and Ear Infirmary,1270,0.0080,2.028989,Mass General Brigham,Boston,42.363000,-71.068800,Boston Metro,0.699062
4446,2020-06-01,Massachusetts Eye and Ear Infirmary,1390,0.0100,1.947998,Mass General Brigham,Boston,42.363000,-71.068800,Boston Metro,0.699062


In [21]:
df_clean = df[df['emergency_department'] != 'All Visits']
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4370 entries, 1 to 4447
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   month                 4370 non-null   datetime64[ns]
 1   emergency_department  4370 non-null   object        
 2   visits                4370 non-null   int64         
 3   pct_monthly_visits    4370 non-null   float64       
 4   average_los_hours     4370 non-null   float64       
 5   hospital_system       4370 non-null   object        
 6   city                  4370 non-null   object        
 7   latitude              4370 non-null   float64       
 8   longitude             4370 non-null   float64       
 9   region                4370 non-null   object        
 10  correlation           4370 non-null   float64       
dtypes: datetime64[ns](1), float64(5), int64(1), object(4)
memory usage: 409.7+ KB


In [22]:
hospital_counts = (
    df.groupby("emergency_department")["month"]
      .count()
      .sort_values()
)

hospital_counts


emergency_department
North Adams Regional Hospital                12
Steward Norwood Hospital                     21
Signature Healthcare Brockton Hospital       61
Nashoba Valley Medical Center                71
Steward Carney Hospital                      71
Lowell General Hospital                      78
Marlborough Hospital                         78
Martha's Vineyard Hospital                   78
Massachusetts Eye and Ear Infirmary          78
Massachusetts General Hospital               78
MelroseWakefield Hospital                    78
Mercy Medical Center                         78
MetroWest Medical Center                     78
Milford Regional Medical Center              78
Morton Hospital                              78
Mount Auburn Hospital                        78
All Visits                                   78
Lawrence General Hospital                    78
Newton-Wellesley Hospital                    78
North Shore Medical Center                   78
Northeast Hospital 

## Export cleaned data

In [23]:
# Save cleaned dataset for analysis notebooks
import os
os.makedirs("data/processed", exist_ok=True)
df_clean.to_csv("data/processed/df_clean.csv", index=False)
print("Saved: data/processed/df_clean.csv")

Saved: data/processed/df_clean.csv
